In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import warnings
warnings.filterwarnings("ignore")


result = pd.read_csv("dataset.csv", sep = ',',engine ='python')

RANDOM_STATE = 42
result['Label_Num'] = result.Label.map({"Afanasyev": 0, "Bobadilla": 1, "Carbunar": 2, "Chen": 3,  
                                        "Clarke": 4, "Finlayson": 5, "He": 6, "Iyengar": 7,
                                        "Lisetti": 8, "Liu": 9, "Narasimhan": 10, "Navlakha": 11,
                                        "Pan": 12, "Pissinou": 13, "Rangaswami": 14, "Rishe": 15,
                                        "Sadjadi": 16, "Saeed": 17, "Smith": 18, "Weiss": 19,
                                        "Xie": 20, "Zeng": 21 })
result.head(20)


In [ ]:
split_list = [2, 3, 4, 5, 6, 7, 8]
classifier_list = ["Multinomial Naive Bayes", "Support Vector Machine", "Decision Tree", "Random Forest"]
X = result['Abstract']
y = result['Label_Num']
vectorizer = CountVectorizer(stop_words='english')

for c in classifier_list:
    fig, ax = plt.subplots()
    ax.set_xlabel("Values of K")
    ax.set_ylabel("Percentage")
    ax.set_title("Accuracy")
    print(c, "Classifier\n")
    for s in split_list:
        print("K = ", s)
        kf = KFold(n_splits=s, shuffle=True, random_state=RANDOM_STATE)
        for train_ind, test_ind in kf.split(X):
            # Split X and y on the index given by the KFold
            X_train, X_test = X[train_ind], X[test_ind]
            y_train, y_test = y[train_ind], y[test_ind]
            # Generate the text counts using the vectorizer
            train_features = vectorizer.fit_transform([r for r in X_train])
            test_features = vectorizer.transform([r for r in X_test])
            # Fit model to the training data
            if(c == "Multinomial Naive Bayes"):
                classifier = MultinomialNB(alpha = 1.0, fit_prior = False, class_prior = None)
            elif(c == "Support Vector Machine"):
                classifier = SVC(kernel = 'linear', random_state = 0)
            elif(c == "Random Forest"):
                classifier = RandomForestClassifier(n_estimators = 15 ,criterion = 'entropy', random_state = 0)
            else:
                classifier = tree.DecisionTreeClassifier(class_weight=None ,criterion = 'entropy', random_state = 0)
            classifier.fit(train_features, [int(r) for r in y_train])
        scores = cross_val_score(classifier, test_features, y_test, cv=s)
        avg = scores.mean()
        std = scores.std()
        ax.scatter(s, avg)
        ax.errorbar(s, avg, yerr=std)
        print("Accuracy: %0.2f ( +/- %0.2f)\n" % (avg, std * 2))
    plt.show()
    print("="*110)

